# Some baseline models (2)
*Anders Poirel - 13-02-2020*


In this notebook I will build a few simple models to get a feel for the dataset, and ideas on directions to improve the competition score in the future.

In [ ]:
**Note**: I realized I used *median* absolute error instead of mean absolute error in a previous version of this notebook, which explain why my CV scores were so far from the test set scores!

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from os.path import join

In [2]:
DATA_PATH = '../data/raw/'

## Acquiring the data

In [3]:
X_test = pd.read_csv(join(DATA_PATH, 'dengue_features_test.csv'))
X_train = pd.read_csv(join(DATA_PATH, 'dengue_features_train.csv'))
y_train = pd.read_csv(join(DATA_PATH, 'dengue_labels_train.csv'))

### Encodings

In [4]:
X_train = pd.get_dummies(X_train, columns = ['city'], drop_first = True)
X_test = pd.get_dummies(X_test, columns = ['city'], drop_first = True)

### Dropping unecessary / correlated columns

In [5]:
X_train = X_train.drop('week_start_date', axis = 1)
X_test = X_test.drop('week_start_date', axis = 1)
y_train = y_train['total_cases']

## Building the models

Now can move on to building a few simple models. We will be evaluating on Mean Absolute Error (MAE) since this is how the competition will be scored.

**Note**: at first I observer unusually good MAE scores on CV, which turned out to be spurious since I was using a normal cross-validation split and indeed fitting the model to future data, so make sure to use a `TimeSeriesSplit` in your CV 🙂

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.model_selection import (cross_validate, TimeSeriesSplit, 
                                     RandomizedSearchCV)

### OLS

In [7]:
imputer = SimpleImputer(strategy = 'median')

In [8]:
ols_model = Pipeline([
    ('impute_m', imputer),
    ('ols', LinearRegression())
])

In [9]:
cv_res = cross_validate(
    estimator = ols_model,
    X = X_train,
    y = y_train,
    cv = TimeSeriesSplit(n_splits = 10),
    scoring = 'neg_mean_absolute_error',
    n_jobs = -1
)
ols_score = np.mean(cv_res['test_score'])

In [10]:
ols_score

-35.796794518364635

Is this overfitting? Let's add some complexity penalties to find out.

### ElasticNet

**Note**: in the first iteration of this test, I forgot to scale features (!)
resulting in an MAE of ~26.4. Scaling improved this a bit

In [12]:
en_model = Pipeline([
    ('scale', StandardScaler()),
    ('impute_m', imputer),
    ('en', ElasticNet())
])

In [13]:
cv_res = cross_validate(
    estimator = en_model,
    X = X_train,
    y = y_train,
    cv = TimeSeriesSplit(n_splits = 10),
    scoring = 'neg_mean_absolute_error',
    n_jobs = -1
)
en_score = np.mean(cv_res['test_score'])

In [14]:
en_score

-28.03091292318651

Let's take a look at where this model is making errors:

FIXME

#### Cranking up the penalty

In [24]:
enmore_model = Pipeline([
    ('scale', StandardScaler()),
    ('impute_m', imputer),
    ('en', ElasticNet(alpha = 2))
])
cv_res = cross_validate(
    estimator = enmore_model,
    X = X_train,
    y = y_train,
    cv = TimeSeriesSplit(n_splits = 10),
    scoring = 'neg_mean_absolute_error',
    n_jobs = -1
)
enmore_score = np.mean(cv_res['test_score'])
enmore_score

-26.557742385277624

Doubling the penalty seems to give a tiny edge over the previous model. Manual testing shows that increasing it further makes things worse.

It looks like raw regularization does not improve things by much, so let's try increasing model complexity

### Polynomial Kernels for Regression

In [25]:
from sklearn.preprocessing import PolynomialFeatures

In [26]:
poly_model = Pipeline([
    ('impute_m', imputer),
    ('scale', StandardScaler()),
    ('poly_f', PolynomialFeatures(degree = 2)),
    ('poly_reg', LinearRegression())
])

In [27]:
cv_res = cross_validate(
    estimator = poly_model,
    X = X_train,
    y = y_train,
    cv = TimeSeriesSplit(n_splits = 10),
    scoring = 'neg_median_absolute_error',
    n_jobs = -1
)
poly_score = np.mean(cv_res['test_score'])

In [28]:
poly_score

-155.23319488400816

#### With complexity penalties

Default:

In [32]:
poly_model_2 = Pipeline([
    ('impute_m', imputer),
    ('scale', StandardScaler()),
    ('poly_f', PolynomialFeatures(degree = 2)),
    ('poly_reg', ElasticNet())
])
cv_res = cross_validate(
    estimator = poly_model_2,
    X = X_train,
    y = y_train,
    cv = TimeSeriesSplit(n_splits = 10),
    scoring = 'neg_mean_absolute_error',
    n_jobs = -1
)
poly_score_2 = np.mean(cv_res['test_score'])

In [33]:
poly_score_2

-39.77474083230882

Increased ($\alpha$ = 5 was obtainied by trying manually all values in 1-10 range and 20, 50, 100)

In [34]:
poly_model_3 = Pipeline([
    ('impute_m', imputer),
    ('scale', StandardScaler()),
    ('poly_f', PolynomialFeatures(degree = 2)),
    ('poly_reg', ElasticNet(alpha = 6))
])
cv_res = cross_validate(
    estimator = poly_model_3,
    X = X_train,
    y = y_train,
    cv = TimeSeriesSplit(n_splits = 10),
    scoring = 'neg_mean_absolute_error',
    n_jobs = -1
)
poly_score_3 = np.mean(cv_res['test_score'])

In [35]:
poly_score_3

-24.209824228207065

Finally a big improvement!

Testing with `degree = 3` worsens performance across the board. I did not bother testing with even higher degrees.

## Take-aways

Adding an (untuned!) ElasticNet penalty improved performance quite a bit, (30.5 -> 23.4 MAE), while increasing the complexity of the model with a polynomial kernel decreased it (30.5 -> 155.2 MAE), which hints that we overfit easily and improvements in the model will more likely come from more regularization than model complexity - Or at least that any increase in complexity will need to be balanced big regularization)

Things to try out in the future:
- some pre-selection of features to eliminate redundant ones 
- model-based selection of features
- looking at where the model is making errors and engineerign features refecting that if we see a trend
- model-based feature imputation
- splitting the model between two cities as the structure of the data is different between them

### Building a submission

The best model we found was a polynomial regression with deg. 2 kernels and a ElasticNet penalty $\alpha = 5$, so I use this for our first submission on DrivenData:

In [167]:
submission = pd.read_csv(join(DATA_PATH, 'submission_format.csv'))

In [171]:
poly_model_3.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('impute_m',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='median',
                               verbose=0)),
                ('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('poly_f',
                 PolynomialFeatures(degree=2, include_bias=True,
                                    interaction_only=False, order='C')),
                ('poly_reg',
                 ElasticNet(alpha=5, copy_X=True, fit_intercept=True,
                            l1_ratio=0.5, max_iter=1000, normalize=False,
                            positive=False, precompute=False, random_state=None,
                            selection='cyclic', tol=0.0001,
                            warm_start=False))],
         verbose=False)

In [185]:
y_pred = poly_model_3.predict(X_test)
submission['total_cases'] = np.round(y_pred).astype(int)

In [186]:
submission

,city,year,weekofyear,total_cases
0,sj,2008,18,23
1,sj,2008,19,22
2,sj,2008,20,27
3,sj,2008,21,26
4,sj,2008,22,26
...,...,...,...,...
411,iq,2013,22,16
412,iq,2013,23,13
413,iq,2013,24,14
414,iq,2013,25,14


In [187]:
submission.to_csv('../models/baseline.csv', index = False)